In [1]:
# Imports 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
# import datasets
df_m = pd.read_csv('./data/PP_males.csv')

df_f = pd.read_csv('./data/PP_females.csv')

In [3]:
df_f.head()

,Place,Div/Tot,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace
0,1,1/86,30,Florence Jepkosgei,22.0,Kenya .,0:32:59,32:58,5:19
1,2,2/86,8,Eunice Chepkirui,21.0,Kenya .,0:33:14,33:13,5:21
2,3,1/247,3200,Elena Orlova,36.0,Gaithersburg MD,0:34:26,34:25,5:33
3,4,1/173,2,Susannah Kvasnicka,34.0,Great Falls VA,0:34:43,34:42,5:35
4,5,1/142,6,Casey Smith,26.0,Arlington VA,0:35:04,35:02,5:39


In [4]:
df_m.shape, df_f.shape

((1265, 9), (1105, 9))

### Dataframe Examination

In [5]:
df_m.isnull().sum(),df_f.isnull().sum()

(Place       0
 Div/Tot     2
 Num         0
 Name        0
 Ag          1
 Hometown    0
 Gun Tim     0
 Net Tim     0
 Pace        0
 dtype: int64, Place       0
 Div/Tot     2
 Num         0
 Name        0
 Ag          1
 Hometown    0
 Gun Tim     0
 Net Tim     0
 Pace        0
 dtype: int64)

#### Remove null values and bad data 

In [6]:
df_m.dropna(axis=0, inplace=True)

In [7]:
df_f.dropna(axis=0, inplace=True)

In [8]:
df_m.Ag.value_counts().sort_index().head()

-1.0     2
 9.0     1
 10.0    1
 11.0    1
 12.0    6
Name: Ag, dtype: int64

In [9]:
df_f.Ag.value_counts().sort_index().head()

-1.0     2
 1.0     1
 10.0    2
 11.0    1
 12.0    2
Name: Ag, dtype: int64

In [10]:
#drop the -1 age variables as they are clearly misrecorded
df_m =df_m[df_m['Ag']>0]
df_f =df_f[df_f['Ag']>0] 

In [11]:
#Examine the divisions
len(df_m['Div/Tot'].unique()),len(df_f['Div/Tot'].unique())

(1261, 928)

** This many divisions doesn't make sense so need to create new divisions **

### Feature Engineering

### Age variable

In [12]:
#Male dataframe minima and maxima
df_m.Ag.min(),df_m.Ag.max()

(9.0, 84.0)

In [13]:
#Female dataframe minima and maxima
df_f.Ag.min(),df_f.Ag.max()

(1.0, 74.0)

In [14]:
# Create an age variable to replace the incorrect data
def define_age(x):
    if x < 15:
        return '14 or less'
    elif x < 20:
        return '15-19'
    elif x < 30:
        return '20-29'
    elif x < 40:
        return '30-39'
    elif x < 50:
        return '40-49'
    elif x < 60:
        return '50-59'
    elif x < 70:
        return '60-69'
    elif x < 80 :
        return '70-79'
    else:
        return 'over 80'

In [15]:
#apply to datasets 
df_m['Division'] = df_m['Ag'].apply(define_age)
df_f['Division'] = df_f['Ag'].apply(define_age)

In [16]:
df_m.sample(5)


,Place,Div/Tot,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace,Division
444,445,86/214,117,John Bosworth Jr.,44.0,Rockville MD,49:26,48:11,7:46,40-49
1083,1084,22/28,1725,Timothy Lim,12.0,Rockville MD,1:05:48,1:00:44,9:47,14 or less
157,158,22/143,1790,Charles Majewski,32.0,Gaithersburg MD,43:11,42:08,6:47,30-39
432,433,28/48,811,Dan Coghlan,24.0,Leesburg VA,51:20,47:59,7:44,20-29
431,432,83/214,199,William Rowell,40.0,Gaithersburg MD,51:49,47:56,7:43,40-49


In [17]:
df_f.sample(5)

,Place,Div/Tot,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace,Division
243,689,161/247,119,Dawn Brandley,35.0,Springfield VA,1:03:38,59:54,9:39,30-39
530,969,127/142,1146,April Freeburn,29.0,Centreville VA,1:14:09,1:08:56,11:06,20-29
826,191,28/142,2076,Amy Ostrow,26.0,Gaithersburg MD,53:01,49:50,8:02,20-29
627,1083,168/173,2219,Althea Raiford,34.0,Silver Spring M,1:25:50,1:22:05,13:13,30-39
642,1097,170/173,3353,Karen Liu,33.0,Potomac MD,1:37:54,1:34:31,15:13,30-39


In [18]:
df_m['Division'].value_counts().sort_index()

14 or less     26
15-19          45
20-29         144
30-39         346
40-49         415
50-59         205
60-69          69
70-79           9
over 80         2
Name: Division, dtype: int64

In [19]:
df_f['Division'].value_counts().sort_index()

14 or less     13
15-19          26
20-29         228
30-39         420
40-49         283
50-59         104
60-69          24
70-79           3
Name: Division, dtype: int64

### Temporal variables

In [20]:
# create a function to remove trailing zeros from time variables
def remove_zero(X):
    if X.startswith(("2","3","5","4")):
        return X[:-3].strip()
    else: 
        return X

In [21]:
#create a function to add a leading hour markers for the rows without hours
def add_zero(X):
    if X.startswith(("2","3","5","4")):
        return "0:"+X
    else: 
        return X

In [22]:
df_f['Gun Tim1'] = df_f['Gun Tim'].map(add_zero)
df_f['Net Tim1']=df_f['Net Tim'].map(add_zero)
df_f.sample(10)

,Place,Div/Tot,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace,Division,Gun Tim1,Net Tim1
570,1043,165/173,839,Linda Consorti,42.0,Hyattsville MD,1:17:30,1:13:50,11:53,40-49,1:17:30,1:13:50
102,586,26/76,1383,Ann Hoffman,50.0,Rockville MD,1:01:28,58:25,9:24,50-59,1:01:28,0:58:25
329,819,137/173,1946,Lisa Moeller,43.0,Gaithersburg MD,1:06:31,1:03:16,10:11,40-49,1:06:31,1:03:16
762,123,16/173,1285,Amy Greene,31.0,Rockville MD,50:37,47:34,7:40,30-39,0:50:37,0:47:34
767,175,11/26,3329,Perri Miller,15.0,Potomac MD,50:41,49:17,7:56,15-19,0:50:41,0:49:17
557,1004,17/19,1953,Shirley Moore,63.0,Gaithersburg MD,1:16:13,1:11:02,11:26,60-69,1:16:13,1:11:02
922,296,65/247,2836,Erica Yuan,38.0,Kensington MD,56:08,52:58,8:32,30-39,0:56:08,0:52:58
1034,420,60/142,1349,Nora Hattery,26.0,Baltimore MD,58:30,55:13,8:54,20-29,0:58:30,0:55:13
73,522,86/173,1824,Brauna Martin,41.0,Potomac MD,1:00:46,57:00,9:11,40-49,1:00:46,0:57:00
702,92,16/173,2994,Heather Chen-Mayer,44.0,Garret Park MD,0:47:46,46:31,7:30,40-49,0:47:46,0:46:31


In [23]:
df_m['Gun Tim1'] = df_m['Gun Tim'].map(add_zero)
df_m['Net Tim1']=df_m['Net Tim'].map(add_zero)
df_m.sample(5)

,Place,Div/Tot,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace,Division,Gun Tim1,Net Tim1
908,909,169/214,1224,John Gilbert,42.0,Germantown MD,1:00:11,56:56,9:10,40-49,1:00:11,0:56:56
219,220,41/214,102,Luis Amaya,43.0,Arlington VA,44:56,43:39,7:02,40-49,0:44:56,0:43:39
868,869,164/214,1182,David Gamzon,42.0,Rockville MD,59:18,55:59,9:01,40-49,0:59:18,0:55:59
1220,1221,209/214,1902,William Mease,40.0,Boyds MD,1:16:47,1:11:43,11:33,40-49,1:16:47,1:11:43
1101,1102,57/73,2839,Bob Zachariasiewi,59.0,Rockville MD,1:05:16,1:01:51,9:58,50-59,1:05:16,1:01:51


In [24]:
#format the timeframe columns  as a datetime object for further manipulation
df_f['Net_Time1'] =pd.to_datetime(df_f['Net Tim1'],format='%H:%M:%S')
df_f['Gun_Time1'] =pd.to_datetime(df_f['Gun Tim1'],format='%H:%M:%S')

df_m['Net_Time1'] =pd.to_datetime(df_m['Net Tim1'],format='%H:%M:%S')
df_m['Gun_Time1'] =pd.to_datetime(df_m['Gun Tim1'],format='%H:%M:%S')

In [25]:
df_m.sample(5)

,Place,Div/Tot,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace,Division,Gun Tim1,Net Tim1,Net_Time1,Gun_Time1
1189,1190,137/143,699,Rahmon Burton,32.0,Silver Spring MD,1:10:57,1:07:45,10:55,30-39,1:10:57,1:07:45,1900-01-01 01:07:45,1900-01-01 01:10:57
47,48,7/45,2637,John O Thornton,17.0,Columbia MD,36:10,36:07,5:49,15-19,0:36:10,0:36:07,1900-01-01 00:36:07,1900-01-01 00:36:10
251,252,37/203,545,Juan Balsa,39.0,Washington DC,45:36,44:22,7:09,30-39,0:45:36,0:44:22,1900-01-01 00:44:22,1900-01-01 00:45:36
620,621,76/143,2701,Ganesh Vishwanathan,30.0,Germantown MD,52:40,51:15,8:15,30-39,0:52:40,0:51:15,1900-01-01 00:51:15,1900-01-01 00:52:40
1105,1106,182/203,3324,Mo Fathelbab,39.0,Alexandria VA,1:03:37,1:02:15,10:02,30-39,1:03:37,1:02:15,1900-01-01 01:02:15,1900-01-01 01:03:37


In [26]:
# Calculate the time difference between gun time and net time
df_f['Gun_Net_Diff'] =df_f['Gun_Time1']- df_f['Net_Time1'] 
df_f.sample(5)

,Place,Div/Tot,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace,Division,Gun Tim1,Net Tim1,Net_Time1,Gun_Time1,Gun_Net_Diff
371,785,184/247,2344,Sandra Rye,37.0,Pomfret MD,1:07:25,1:02:17,10:02,30-39,1:07:25,1:02:17,1900-01-01 01:02:17,1900-01-01 01:07:25,00:05:08
1101,607,100/173,992,Dana Dosik,40.0,Silver Spring M,59:54,58:44,9:27,40-49,0:59:54,0:58:44,1900-01-01 00:58:44,1900-01-01 00:59:54,00:01:10
262,727,111/173,935,Diana Desantolo,33.0,Gaithersburg MD,1:04:13,1:00:37,9:46,30-39,1:04:13,1:00:37,1900-01-01 01:00:37,1900-01-01 01:04:13,00:03:36
3,4,1/173,2,Susannah Kvasnicka,34.0,Great Falls VA,0:34:43,34:42,5:35,30-39,0:34:43,0:34:42,1900-01-01 00:34:42,1900-01-01 00:34:43,00:00:01
73,522,86/173,1824,Brauna Martin,41.0,Potomac MD,1:00:46,57:00,9:11,40-49,1:00:46,0:57:00,1900-01-01 00:57:00,1900-01-01 01:00:46,00:03:46


In [27]:
df_m['Gun_Net_Diff'] =df_m['Gun_Time1']- df_m['Net_Time1'] 
df_m.sample(5)

,Place,Div/Tot,Num,Name,Ag,Hometown,Gun Tim,Net Tim,Pace,Division,Gun Tim1,Net Tim1,Net_Time1,Gun_Time1,Gun_Net_Diff
849,850,9/19,3218,Roger Burkhart,68.0,Gaithersburg MD,59:01,55:42,8:58,60-69,0:59:01,0:55:42,1900-01-01 00:55:42,1900-01-01 00:59:01,00:03:19
256,257,49/214,1684,Leonard Lee,40.0,Silver Spring MD,45:37,44:28,7:10,40-49,0:45:37,0:44:28,1900-01-01 00:44:28,1900-01-01 00:45:37,00:01:09
1000,1001,106/132,1709,Mark Levitt,50.0,Bethesda MD,59:53,58:37,9:26,50-59,0:59:53,0:58:37,1900-01-01 00:58:37,1900-01-01 00:59:53,00:01:16
1110,1111,197/214,1248,Ed Goldman,43.0,Frederick MD,1:06:30,1:02:42,10:06,40-49,1:06:30,1:02:42,1900-01-01 01:02:42,1900-01-01 01:06:30,00:03:48
454,455,73/203,1862,Keith Mccracken,36.0,Chantilly VA,49:47,48:27,7:48,30-39,0:49:47,0:48:27,1900-01-01 00:48:27,1900-01-01 00:49:47,00:01:20


In [28]:
# seperate the divisions and totals
df_f[['Div','Tot']] = df_f['Div/Tot'].str.split('/', n=1, expand=True)
df_m[['Div','Tot']] = df_m['Div/Tot'].str.split('/', n=1, expand=True)

In [29]:
df_f.columns

Index(['Place', 'Div/Tot', 'Num', 'Name', 'Ag', 'Hometown', 'Gun Tim',
       'Net Tim', 'Pace', 'Division', 'Gun Tim1', 'Net Tim1', 'Net_Time1',
       'Gun_Time1', 'Gun_Net_Diff', 'Div', 'Tot'],
      dtype='object')

In [30]:
#analyze the composition of each division
df_f.shape[0]/len(sorted(df_f.Div.unique())), df_m.shape[0]/len(sorted(df_m.Div.unique()))

(4.4574898785425106, 5.892523364485982)

In [31]:
#rename the columns 
df_f.rename(columns={'Num':'Number','Ag':'Age','Gun Tim1':'Gun_Time',
                     'Net Tim1':'Net_Time','Tot':'Total_Racers'},
            inplace=True)
df_m.rename(columns={'Num':'Number','Ag':'Age','Gun Tim1':'Gun_Time',
                     'Net Tim1':'Net_Time','Tot':'Total_Racers'},
            inplace=True)

In [32]:
# Assign columns to correct data types
to_o =['Place','Div','Number']
to_num=['Total_Racers','Pace']
df_f[to_o]= df_f[to_o].astype(str)
df_m[to_o]= df_m[to_o].astype(str)


df_f['Total_Racers']= df_f['Total_Racers'].astype(int)
df_m['Total_Racers']= df_m['Total_Racers'].astype(int)



df_f['Pace']= df_f['Pace'].str.replace(":",'.')
df_f['Pace']= df_f['Pace'].astype(float)

df_m['Pace']= df_m['Pace'].str.replace(":",'.')
df_m['Pace']= df_m['Pace'].astype(float)

In [33]:
#create an unique marker for each gender when dataframes are combined
df_m['Gender'] = 'male'
df_f['Gender'] = 'female'

In [34]:
# create a combined dataframe for future use
df_total = pd.concat([df_m, df_f])
df_total.sample(10)

,Place,Div/Tot,Number,Name,Age,Hometown,Gun Tim,Net Tim,Pace,Division,Gun_Time,Net_Time,Net_Time1,Gun_Time1,Gun_Net_Diff,Div,Total_Racers,Gender
1060,426,103/247,3307,Lesley Green,36.0,Sterling VA,59:01,55:24,8.55,30-39,0:59:01,0:55:24,1900-01-01 00:55:24,1900-01-01 00:59:01,00:03:37,103,247,female
949,950,152/201,2462,Alan Sims,49.0,Gaithersburg MD,1:03:08,57:49,9.19,40-49,1:03:08,0:57:49,1900-01-01 00:57:49,1900-01-01 01:03:08,00:05:19,152,201,male
276,636,149/247,912,Ann Dayton,37.0,New Market MD,1:04:27,59:08,9.31,30-39,1:04:27,0:59:08,1900-01-01 00:59:08,1900-01-01 01:04:27,00:05:19,149,247,female
15,16,2/26,3266,Amanda Robotti,18.0,College Park MD,0:39:39,39:34,6.23,15-19,0:39:39,0:39:34,1900-01-01 00:39:34,1900-01-01 00:39:39,00:00:05,2,26,female
1039,1040,166/201,1047,Paul Englehart,49.0,Derwood MD,1:03:03,59:37,9.36,40-49,1:03:03,0:59:37,1900-01-01 00:59:37,1900-01-01 01:03:03,00:03:26,166,201,male
773,774,151/214,134,Lake Coulson,41.0,Gaithersburg MD,55:07,53:48,8.40,40-49,0:55:07,0:53:48,1900-01-01 00:53:48,1900-01-01 00:55:07,00:01:19,151,214,male
1196,1197,198/203,950,Donald Dietrichson,39.0,Silver Spring MD,1:09:12,1:08:05,10.58,30-39,1:09:12,1:08:05,1900-01-01 01:08:05,1900-01-01 01:09:12,00:01:07,198,203,male
56,57,9/96,1548,Kevin Kemp,27.0,Reston VA,37:06,37:03,5.58,20-29,0:37:06,0:37:03,1900-01-01 00:37:03,1900-01-01 00:37:06,00:00:03,9,96,male
40,41,6/96,3346,Ted Bradley,27.0,Comus MD,35:37,35:33,5.44,20-29,0:35:37,0:35:33,1900-01-01 00:35:33,1900-01-01 00:35:37,00:00:04,6,96,male
392,393,52/143,2125,David Peikin,34.0,Gaithersburg MD,48:21,47:05,7.35,30-39,0:48:21,0:47:05,1900-01-01 00:47:05,1900-01-01 00:48:21,00:01:16,52,143,male


In [35]:
# pickle dataframes for future use 
df_f.to_pickle('female.pkl')
df_m.to_pickle('male.pkl')

df_total.to_pickle('total.pkl')